# 1. Procesado de datos

In [9]:
import pandas as pd
import numpy as np

df = pd.read_csv('./data/train.csv')

In [12]:
import spacy

nlp = spacy.load('en_core_web_lg')

df

,label,text,id
0,0,"Hi Roy hope you are ok, Trans people are not g...",95e98db99c2
1,0,But fuckin' hell what even is biology,a87b8708e63
2,0,Whose the nice looking clergyman?,5b3cb03803f
3,1,"AIDS ARE IN YOUR WAY, SIN HAS CONSEQUENCES AND...",5b0bad2347e
4,0,to learn,332048bd188
...,...,...,...
8143,1,Yeah...if we still alive at the time of the ne...,9141edf7189
8144,0,There is something fundamentally wrong with this,fde30735725
8145,1,"This always confused me. If homosexuality is ""...",3d3456b6659
8146,1,disgusting,eace075fe4d


In [3]:
en_stopwords = nlp.Defaults.stop_words

def remove_stop_words(text):
    cleanText = ''
    phrase = nlp(text)
    for token in phrase:
        if not token.is_stop and not token.is_punct and not token.like_url:
            cleanText += ' ' + token.text

    return cleanText

df['text_cleaned'] = df['text'].apply(remove_stop_words)

In [4]:
df['text_cleaned']

0        Hi Roy hope ok Trans people gay thing s ramme...
1                                     fuckin hell biology
2                                  nice looking clergyman
3                           AIDS WAY SIN CONSEQUENCES BAD
4                                                   learn
                              ...                        
8143     Yeah alive time election happen fairly soon U...
8144                                  fundamentally wrong
8145     confused homosexuality big deal proud normal ...
8146                                           disgusting
8147     Peter Sørensen note Peter poor maths 13 27 eq...
Name: text_cleaned, Length: 8148, dtype: object

Separamos el conjunto en entrenamiento y test.

In [5]:
# spling training and testing
from sklearn.model_selection import train_test_split

X = df['text_cleaned']
y = df['label']

#train
X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

# Define el pipeline
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LinearSVC())
])

# Define los parámetros para la búsqueda de cuadrícula de TfidfVectorizer
parameters_tfidf = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'tfidf__token_pattern': (r"\b\w\w+\b", r"\b[a-zA-Z]{3,}\b"),
    'tfidf__norm': ('l1', 'l2', None),
    'tfidf__use_idf': (True, False),
    'tfidf__sublinear_tf': (True, False)
}

# Realiza la búsqueda de cuadrícula para TfidfVectorizer
grid_search_tfidf = GridSearchCV(text_clf, parameters_tfidf, cv=5, n_jobs=-1)
grid_search_tfidf.fit(X_train, y_train)

# Obtén los mejores parámetros encontrados para TfidfVectorizer
best_params_tfidf = grid_search_tfidf.best_params_

# Define los parámetros para la búsqueda de cuadrícula de LinearSVC
parameters_svc = {
    'clf__penalty': ('l1', 'l2'),
    'clf__loss': ('hinge', 'squared_hinge'),
    'clf__dual': (True, False),
    'clf__tol': (1e-4, 1e-3),
    'clf__C': (0.1, 1, 10),
    'clf__multi_class': ('ovr', 'crammer_singer'),
    'clf__fit_intercept': (True, False),
    'clf__intercept_scaling': (1.0, 2.0),
    'clf__class_weight': (None, 'balanced'),
    'clf__verbose': (0, 1),
    'clf__random_state': (None, 42),
    'clf__max_iter': (1000, 2000)
}

# Actualiza el pipeline con los mejores parámetros de TfidfVectorizer
text_clf.set_params(**best_params_tfidf)

# Realiza la búsqueda de cuadrícula para LinearSVC utilizando los mejores parámetros de TfidfVectorizer
grid_search_svc = GridSearchCV(text_clf, parameters_svc, cv=5, n_jobs=-1)
grid_search_svc.fit(X_train, y_train)

# Muestra los mejores parámetros encontrados para LinearSVC
print("Mejores parámetros para LinearSVC:")
print(grid_search_svc.best_params_)

C:\Users\ivana\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [ ]:
best_params_svc = grid_search_svc.best_params_

# Crea una nueva pipeline con los mejores parámetros encontrados
best_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LinearSVC())
])

best_pipeline.set_params(**best_params_tfidf)
best_pipeline.set_params(**best_params_svc)

best_pipeline.fit(X_train, y_train)

predictions = best_pipeline.predict(X_test)

print(predictions)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [13]:
print(confusion_matrix(y_test, predictions))

[[1457  243]
 [ 453  536]]


In [14]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.76      0.86      0.81      1700
           1       0.69      0.54      0.61       989

    accuracy                           0.74      2689
   macro avg       0.73      0.70      0.71      2689
weighted avg       0.74      0.74      0.73      2689



In [15]:
from sklearn import metrics
metrics.accuracy_score(y_test, predictions)

0.7411677203421346

In [16]:
best_pipeline.predict(["Kill them where they lie ...kids as well"])

NotFittedError: The TF-IDF vectorizer is not fitted